In [1]:
from async_geotiff import GeoTIFF
from obstore.store import S3Store

from lonboard import Map, RasterLayer

In [2]:
BUCKET_URL = "https://ds-wheels.s3.us-east-1.amazonaws.com"
COG_URL = "m_4007307_sw_18_060_20220803.tif"

In [3]:
store = S3Store.from_url(BUCKET_URL, skip_signature=True)

In [4]:
geotiff = await GeoTIFF.open(COG_URL, store=store)

In [5]:
import io

import numpy as np
from async_geotiff import Tile as GeoTIFFTile
from PIL import Image

from lonboard.raster import EncodedImage


def render_rgb_tile(tile: GeoTIFFTile) -> EncodedImage:
    # data is (bands, height, width), uint8
    arr = tile.array.data  # shape: (3, H, W)
    img_arr = np.moveaxis(arr, 0, -1)  # -> (H, W, 3)
    img = Image.fromarray(img_arr, mode="RGB")
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return EncodedImage(data=buf.getvalue(), mime_type="image/png")

In [6]:
layer = RasterLayer.from_async_geotiff(geotiff, render=render_rgb_tile)

In [7]:
m = Map(layer, height=800)

In [8]:
m